<a href="https://www.kaggle.com/code/dascient/kepler-exoplanet-target-classification-model?scriptVersionId=122849595" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
# lets begin!
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df = pd.read_csv('/kaggle/input/kepler-exoplanet-search-results/cumulative.csv',header=0)

/kaggle/input/kepler-exoplanet-search-results/cumulative.csv


In [3]:
df

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559,9560,10031643,K07984.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,0,0,...,-152.0,4.296,0.231,-0.189,1.088,0.313,-0.228,298.74921,46.973351,14.478
9560,9561,10090151,K07985.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,1,...,-166.0,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082
9561,9562,10128825,K07986.01,NaN,CANDIDATE,CANDIDATE,0.497,0,0,0,...,-220.0,4.444,0.056,-0.224,1.031,0.341,-0.114,286.50937,47.163219,14.757
9562,9563,10147276,K07987.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.021,0,0,1,...,-236.0,4.447,0.056,-0.224,1.041,0.341,-0.114,294.16489,47.176281,15.385


In [4]:
# crucial encoding
lb_make = LabelEncoder()
columns = df.columns.values.tolist()
df_encoded = df[columns].copy()

# categorize/encode
for i in columns:
    df_encoded[i] = lb_make.fit_transform(df[i])

# encoded
df_encoded.head()

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,7098,1080,1035,1,0,649,0,0,0,...,330,1176,59,339,503,89,1335,3754,7078,3074
1,1,7098,1081,1036,1,0,618,0,0,0,...,330,1176,59,339,503,89,1335,3754,7078,3074
2,2,7112,1082,2294,2,1,0,0,1,0,...,235,1253,39,259,444,217,1318,6676,7075,3159
3,3,7132,1083,2294,2,1,0,0,1,0,...,237,1273,48,267,367,185,1329,880,7158,3307
4,4,7136,1084,1867,1,0,649,0,0,0,...,200,1147,65,225,622,318,1263,1991,7127,3226


In [5]:
df_encoded.sample(1000)

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
803,803,5072,1782,2294,2,1,0,0,1,1,...,219,684,251,321,1455,433,781,4665,5064,1343
4935,4935,6351,4462,2294,0,0,645,0,0,0,...,335,968,156,327,768,161,1200,6930,6283,2210
1931,1931,5592,3085,213,1,0,648,0,0,0,...,245,1275,31,282,405,172,1321,2159,5596,3462
1964,1964,2505,2892,134,1,0,629,0,0,0,...,214,1230,47,227,482,263,1303,4540,2511,2346
8361,8361,2488,7917,2294,2,1,0,1,0,0,...,333,873,99,319,1146,275,1184,3478,2525,1512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2799,2799,5232,3977,2294,0,0,633,0,0,0,...,323,1006,114,315,788,183,1233,2965,5259,2827
6296,6296,1447,4329,2294,2,1,0,0,1,1,...,203,1164,144,285,413,138,1293,5708,1457,3025
2536,2536,5196,3631,420,1,0,649,0,0,0,...,251,1367,58,408,171,26,1324,5058,5196,3740
1947,1947,7494,2980,1424,1,0,649,0,0,0,...,332,1044,93,325,735,174,1253,6594,7471,2606


In [ ]:
     # pairplot
target = 'koi_disposition'
#sns.pairplot(df_encoded.sample(int(len(df_encoded/1000))),hue=target)
sns.pairplot(df_encoded.sample(10),hue=target)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15, 10)

corr = df_encoded.corr()
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, linewidths=.01, cmap="Purples")
plt.show()

In [ ]:
# user can input choice target
#target = input("target:")

X = df_encoded.dropna().drop(columns=[target]).copy()
y = df_encoded.dropna()[target].ravel().copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_test.info()

In [ ]:
# classifier
#clf = RandomForestClassifier(max_depth=5, n_estimators=200, random_state=42).fit(X_train, y_train)
#clf = AdaBoostClassifier(n_estimators=200, random_state=42).fit(X_train, y_train)
clf = ExtraTreesClassifier(n_estimators=200, random_state=42).fit(X_train, y_train)
#clf = KNeighborsClassifier().fit(X_train, y_train)
#clf = MLPClassifier(alpha=1, max_iter=500).fit(X_train, y_train)

# results
train_predications = clf.predict(X_train)
train_score = clf.score(X_train, y_train)
predictions = clf.predict(X_test)
score = clf.score(X_test, y_test)
train_matrix = confusion_matrix(y_train, train_predications)
test_matrix = confusion_matrix(y_test, predictions)
print("Target:",target)
print("TRAIN SCORE:",train_score)
print("TEST SCORE:",score)

In [ ]:
matrix = pd.DataFrame(test_matrix)

matrix.style.background_gradient(cmap ='Spectral')\
        .set_properties(**{'font-size': '15px'})

In [ ]:
# noise
pd.DataFrame(y_test-clf.predict(X_test),columns=['Miscalculations']).plot(figsize=(20,5),c='purple')

In [ ]:
import time

# iterator
def classification_feat_importance(df_encoded):
    
    df_encoded = df_encoded.drop(columns=['capital-gain','capital-loss'])
    
    # iterate through each column variable as classification targets
    for target in df_encoded.columns.values:
        X = df_encoded.dropna().drop(columns=[target]).copy()
        y = df_encoded.dropna()[target].ravel().copy()

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        
        time.sleep(30)
        
        # classifiers
        #clf1 = RandomForestClassifier(max_depth=5, n_estimators=200, random_state=42).fit(X_train, y_train)
        clf2 = AdaBoostClassifier(n_estimators=200, random_state=42).fit(X_train, y_train)
        #clf3 = ExtraTreesClassifier(n_estimators=200, random_state=42).fit(X_train, y_train)
        #clf4 = KNeighborsClassifier().fit(X_train, y_train)
        #clf5 = MLPClassifier(alpha=1, max_iter=500).fit(X_train, y_train)
        classifiers = [
                       #clf1, 
                       clf2, 
                       #clf3, 
                       #clf4, 
                       #clf5
                      ]

        for classifier in classifiers:
            results = []
            #test_matrix = confusion_matrix(y_test, clf.predict(X_test))
            results.append({"classifier":str(classifier).split("(")[0],"target":target,"test_score":classifier.score(X_test, y_test)})
            print("Classifier:",str(classifier).split("(")[0],"\t\tTarget:",target,"\tScore:",classifier.score(X_test, y_test))
            
    return pd.DataFrame(results)

In [ ]:
classification_feat_importance(df_encoded)

In [ ]:
# choose target variable
#target = input("Enter target variable: ")
target = "koi_disposition"

# quick proof of concept
a = df.copy()

# find random sample
from random import randrange
idx = randrange(len(a))

# print random configuration item
b = pd.DataFrame(a.loc[idx]).T
print(f"{target}:",b.reset_index()[target][0])

# store sol'n
solution = b.reset_index()[target][0]

# print data point
b